# LAB04 - 2D Convolution

## Part 1 - padding / unpadding

Objectives:
- add some padding to an image by using the following modes :zero, replicate, mirror and circular,
- remove padding of an image.

## Imported librairies and configurations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import skimage as sk
from scipy import ndimage
from scipy import signal
import scipy.stats as st
from scipy import fftpack

from sys import path
path.append('../tisLib')
import myTisLib


## Global parameters for the lab
|parameter|value|description|
|---|---|---|
|NUM_GRAYS| 256| number of gray levels in a gray image|
|MIN_GRAY| 0| minimum value for the gray level (black pixel) |
|MAX_GRAY| 255| maximum value for the gray level (white pixel) |

In [2]:
NUM_GRAYS=256
MIN_GRAY=0
MAX_GRAY=255

# Ex1 - image padding

## Ex1.1 - function to pad an image:

### Function `imagePadding`

Prototype: `def imagePadding(f, mask, type='mirror')`

This function pads the given image `f` with a border related to the `mask` size, by using the technique `type` and returns the result as a 2D array.
> - `f` is a gray image
> - `mask` is the matrix used for padding size computation
> - `type` is the type of padding: `mirror` (default),  `zero`, `replicate` or `circular`

It may be useful to create sub-functions like:

- `circularPadding(f,mask)`
- `mirrorPadding(f,mask)`
- `replicatePadding(f,mask)`
- `zeroPadding(f,mask)`


In [3]:
def circularPadding(f,m):
    if not np.shape(m)[0] % 2 or not np.shape(m)[1] % 2:
        raise Exception("Mask must have odd sizes")
    a = (np.shape(m)[0] - 1) // 2
    b = (np.shape(m)[1] - 1) // 2

    fp = np.pad(f, ((a,a),(b,b),(0,0)) if len(f.shape) == 3 else ((a,a),(b,b)), mode='wrap')
    return fp

def mirrorPadding(f,m):
    if not np.shape(m)[0] % 2 or not np.shape(m)[1] % 2:
        raise Exception("Mask must have odd sizes")
        
    a = (np.shape(m)[0] - 1) // 2
    b = (np.shape(m)[1] - 1) // 2

    fp = np.pad(f, ((a,a),(b,b),(0,0)) if len(f.shape) == 3 else ((a,a),(b,b)), 'symmetric')
    return fp

def replicatePadding(f,m):
    if not np.shape(m)[0] % 2 or not np.shape(m)[1] % 2:
        raise Exception("Mask must have odd sizes")
        
    a = (np.shape(m)[0] - 1) // 2
    b = (np.shape(m)[1] - 1) // 2

    fp = np.pad(f, ((a,a),(b,b),(0,0)) if len(f.shape) == 3 else ((a,a),(b,b)), 'edge')
    return fp

def zeroPadding(f,m):
    if not np.shape(m)[0] % 2 or not np.shape(m)[1] % 2:
        raise Exception("Mask must have odd sizes")
    a = (np.shape(m)[0] - 1) // 2
    b = (np.shape(m)[1] - 1) // 2
    
    if len(np.shape(f)) == 3:   # not a grayscale image
        fp = np.zeros((np.shape(f)[0] + 2 * a, np.shape(f)[1] + 2 * b, np.shape(f)[2]), dtype = f.dtype)
    else:   # grayscale image
        fp = np.zeros((np.shape(f)[0] + 2 * a, np.shape(f)[1] + 2 * b), dtype = f.dtype)

    fp[a:-a,b:-b] = f
    if len(f.shape) == 3 and f.shape[2] == 4:    # Image with alpha -> setting alpha to 1
        fp[0:a,:,3] = 255 if f.dtype == 'uint8' else 1.0
        fp[-a:,:,3] = 255 if f.dtype == 'uint8' else 1.0
        fp[:,0:b,3] = 255 if f.dtype == 'uint8' else 1.0
        fp[:,-b:,3] = 255 if f.dtype == 'uint8' else 1.0
    return fp


def imagePadding(_f, _mask, _type='mirror'):
    if _type == "replicate":
        fp = replicatePadding(_f, _mask)
    elif _type == "mirror":
        fp = mirrorPadding(_f, _mask)
    elif _type == "zero":
        fp = zeroPadding(_f, _mask)
    elif _type == "circular":
        fp = circularPadding(_f, _mask)
    else:
        raise Exception("Unknown padding type")
    return fp



## Test of the `imagePadding` function

This cell creates an array with values (2x3) and computes its symmetric padding with a 5x5 mask.
It compares the two results using the `np.testing.assert_equal` function which generate an error if the test fails.

In [4]:
f=np.array([ 
    [ 1, 2, 3],
    [ 5, 6, 7]
           ])
mask=np.zeros( (5,5) ); 


# TEST MIRROR PADDING

theoreticalMirrorPaddedF=np.array([
    [6, 5, 5, 6, 7, 7, 6],
    [2, 1, 1, 2, 3, 3, 2],
    [2, 1, 1, 2, 3, 3, 2],
    [6, 5, 5, 6, 7, 7, 6],
    [6, 5, 5, 6, 7, 7, 6],
    [2, 1, 1, 2, 3, 3, 2]])

practicalPaddedF=imagePadding(f,mask,'mirror')

# compare results 
np.testing.assert_equal(theoreticalMirrorPaddedF, practicalPaddedF)
print("TEST mirror padding OK")

# TEST ZERO PADDING

theoreticalZeroPaddedF=np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 2, 3, 0, 0],
    [0, 0, 5, 6, 7, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]])

practicalPaddedF=imagePadding(f,mask,'zero')

# compare results 
np.testing.assert_equal(theoreticalZeroPaddedF, practicalPaddedF)
print("TEST zero padding OK")

# TEST REPLICATE PADDING

theoreticalReplicatePaddedF=np.array([
    [1, 1, 1, 2, 3, 3, 3],
    [1, 1, 1, 2, 3, 3, 3],
    [1, 1, 1, 2, 3, 3, 3],
    [5, 5, 5, 6, 7, 7, 7],
    [5, 5, 5, 6, 7, 7, 7],
    [5, 5, 5, 6, 7, 7, 7]])

practicalPaddedF=imagePadding(f,mask,'replicate')

# compare results 
np.testing.assert_equal(theoreticalReplicatePaddedF, practicalPaddedF)
print("TEST replicate padding OK")

# TEST CIRCULAR PADDING

theoreticalCircularPaddedF=np.array([
    [2, 3, 1, 2, 3, 1, 2],
    [6, 7, 5, 6, 7, 5, 6],
    [2, 3, 1, 2, 3, 1, 2],
    [6, 7, 5, 6, 7, 5, 6],
    [2, 3, 1, 2, 3, 1, 2],
    [6, 7, 5, 6, 7, 5, 6]])

practicalPaddedF=imagePadding(f,mask,'circular');

# compare results 
np.testing.assert_equal(theoreticalCircularPaddedF, practicalPaddedF)
print("TEST circular padding OK")

TEST mirror padding OK
TEST zero padding OK
TEST replicate padding OK
TEST circular padding OK


## Ex1.2 - function to remove the padding in an image:

### Function `imageUnpadding`

Prototype: `def imageUnpadding(f, mask)`

This function unpads the given image `f` with a border related to the `mask` size and returns the result as a 2D array.
> - `f` is a gray image
> - `mask` is the matrix used for padding size computation



In [5]:
def imageUnpadding(f,mask):
    if not np.shape(mask)[0] % 2 or not np.shape(mask)[1] % 2:
        raise Exception("Mask must have odd sizes")
    b = (np.shape(mask)[0] - 1) // 2
    a = (np.shape(mask)[1] - 1) // 2

    return f[a:-a, b:-b]


In [6]:
f=np.array([
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 2, 3, 0, 0],
    [0, 0, 5, 6, 7, 0, 0],
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0]])

theoreticalZeroUnpaddedF=np.array([ 
    [ 1, 2, 3],
    [ 5, 6, 7]
           ])

mask=np.zeros( (5,5) ); 

practicalUnpaddedF=imageUnpadding(f,mask)

# compare results 
np.testing.assert_equal(theoreticalZeroUnpaddedF, practicalUnpaddedF)
print("TEST unpadding OK")


TEST unpadding OK
